In [3]:
import pandas as pd
import numpy as np
import os

In [4]:
df = pd.read_json('../output/badm004/badminton004_clip_00000197/alphapose-results.json')

In [5]:
df.head()


,category_id,image_id,keypoints,score
0,1,0.jpg,"[71.23208618164062, 93.92021179199219, 0.04932...",2.488578
1,1,0.jpg,"[183.46817016601562, 58.444889068603516, 0.059...",2.060894
2,1,0.jpg,"[62.8689079284668, 63.7376594543457, 0.2037215...",0.670776
3,1,1.jpg,"[68.7489013671875, 91.97723388671875, 0.305254...",2.425783
4,1,1.jpg,"[182.61886596679688, 57.92866897583008, 0.0728...",1.975177


In [6]:
df.keypoints[0]

[71.23208618164062,
 93.92021179199219,
 0.049321644008159006,
 71.83991241455078,
 92.09673309326172,
 0.06089437007904001,
 79.74166107177734,
 93.31239318847656,
 0.004118522629141,
 73.0555648803711,
 92.70455932617188,
 0.760086238384246,
 78.52600860595703,
 92.09673309326172,
 0.194387748837471,
 71.83991241455078,
 101.82196044921875,
 0.733064293861389,
 82.78079986572266,
 100.60630798339844,
 0.743025302886962,
 69.40860748291016,
 113.97850036621094,
 0.7475318312644951,
 86.4277572631836,
 112.76284790039062,
 0.633324742317199,
 65.15382385253906,
 122.48806762695312,
 0.7638716697692871,
 87.03558349609375,
 123.09590148925781,
 0.582557678222656,
 76.70252990722656,
 124.31155395507812,
 0.6306813955307,
 82.78079986572266,
 123.70372009277344,
 0.61929315328598,
 73.66339874267578,
 143.1541748046875,
 0.7383510470390321,
 77.31035614013672,
 142.5463409423828,
 0.670376896858215,
 87.6434097290039,
 154.702880859375,
 0.681212902069091,
 76.70252990722656,
 158.349838

In [7]:
df[['image_id','score']].head()

,image_id,score
0,0.jpg,2.488578
1,0.jpg,2.060894
2,0.jpg,0.670776
3,1.jpg,2.425783
4,1.jpg,1.975177


In [8]:
coco_format ={   0:"Nose",
    1:"LEye",
    2:"REye",
    3:"LEar",
    4:"REar",
    5:"LShoulder",
    6:  "RShoulder",
    7:  "LElbow",
    8:  "RElbow",
    9:  "LWrist",
    10: "RWrist",
    11: "LHip",
    12: "RHip",
    13: "LKnee",
    14: "Rknee",
    15: "LAnkle",
    16: "RAnkle"}

In [9]:
coco_format_flip = {v:k for k,v in coco_format.items()}

In [10]:
def get_3_tuples(keypoints):
    tuples=[]
    for i in range(int(len(keypoints)/3)):
        tuples.append((keypoints[3*i],keypoints[3*i+1],keypoints[3*i+2]))
    return tuples

In [11]:
get_3_tuples(df.keypoints[0])

[(71.23208618164062, 93.92021179199219, 0.049321644008159006),
 (71.83991241455078, 92.09673309326172, 0.06089437007904001),
 (79.74166107177734, 93.31239318847656, 0.004118522629141),
 (73.0555648803711, 92.70455932617188, 0.760086238384246),
 (78.52600860595703, 92.09673309326172, 0.194387748837471),
 (71.83991241455078, 101.82196044921875, 0.733064293861389),
 (82.78079986572266, 100.60630798339844, 0.743025302886962),
 (69.40860748291016, 113.97850036621094, 0.7475318312644951),
 (86.4277572631836, 112.76284790039062, 0.633324742317199),
 (65.15382385253906, 122.48806762695312, 0.7638716697692871),
 (87.03558349609375, 123.09590148925781, 0.582557678222656),
 (76.70252990722656, 124.31155395507812, 0.6306813955307),
 (82.78079986572266, 123.70372009277344, 0.61929315328598),
 (73.66339874267578, 143.1541748046875, 0.7383510470390321),
 (77.31035614013672, 142.5463409423828, 0.670376896858215),
 (87.6434097290039, 154.702880859375, 0.681212902069091),
 (76.70252990722656, 158.349838

In [12]:
def get_width(keypoints):
    tuples = get_3_tuples(keypoints)
    xs = [i[0] for i in tuples]
    return np.max(xs) - np.min(xs)

In [13]:
def get_height(keypoints):
    tuples = get_3_tuples(keypoints)
    ys = [i[1] for i in tuples]
    return np.max(ys) - np.min(ys)

In [14]:
get_width(df.keypoints[0])

22.489585876464844

In [15]:
get_height(df.keypoints[0])

66.25310516357422

In [16]:
def normalize_tuples(keypoints):
    tuples = get_3_tuples(keypoints)
    xs = [i[0] for i in tuples]
    ys = [i[1] for i in tuples]
    centre = ((np.max(xs)+np.min(xs))/2,(np.max(ys)+np.min(ys))/2)
    for i in range(len(tuples)):
        tuples[i]=(tuples[i][0]-centre[0],tuples[i][1]-centre[1],tuples[i][2])

    return tuples

In [17]:
def normalize_keypoints(keypoints):
    tuples = get_3_tuples(keypoints)
    xs = [i[0] for i in tuples]
    ys = [i[1] for i in tuples]
    centre = ((np.max(xs)+np.min(xs))/2,(np.max(ys)+np.min(ys))/2)
    for i in range(len(tuples)):
        tuples[i]=(tuples[i][0]-centre[0],tuples[i][1]-centre[1],tuples[i][2])

    return [i for tuple in tuples for i in tuple]

In [18]:
normalize_tuples(df.keypoints[0])

[(-5.166530609130859, -31.30307388305664, 0.049321644008159006),
 (-4.558704376220703, -33.12655258178711, 0.06089437007904001),
 (3.3430442810058594, -31.910892486572266, 0.004118522629141),
 (-3.3430519104003906, -32.51872634887695, 0.760086238384246),
 (2.127391815185547, -33.12655258178711, 0.194387748837471),
 (-4.558704376220703, -23.401325225830078, 0.733064293861389),
 (6.382183074951172, -24.61697769165039, 0.743025302886962),
 (-6.990009307861328, -11.24478530883789, 0.7475318312644951),
 (10.02914047241211, -12.460437774658203, 0.633324742317199),
 (-11.244792938232422, -2.735218048095703, 0.7638716697692871),
 (10.636966705322266, -2.1273841857910156, 0.582557678222656),
 (0.3039131164550781, -0.9117317199707031, 0.6306813955307),
 (6.382183074951172, -1.5195655822753906, 0.61929315328598),
 (-2.735218048095703, 17.930889129638672, 0.7383510470390321),
 (0.9117393493652344, 17.323055267333984, 0.670376896858215),
 (11.244792938232422, 29.479595184326172, 0.681212902069091),

In [19]:
a=get_3_tuples(df.keypoints[0])

In [20]:
df['normalized_tuples'] = df.keypoints.apply(normalize_tuples)

In [21]:
df.head()

,category_id,image_id,keypoints,score,normalized_tuples
0,1,0.jpg,"[71.23208618164062, 93.92021179199219, 0.04932...",2.488578,"[(-5.166530609130859, -31.30307388305664, 0.04..."
1,1,0.jpg,"[183.46817016601562, 58.444889068603516, 0.059...",2.060894,"[(-2.8624649047851562, -15.922449111938477, 0...."
2,1,0.jpg,"[62.8689079284668, 63.7376594543457, 0.2037215...",0.670776,"[(-1.2289466857910156, -5.635036468505859, 0.2..."
3,1,1.jpg,"[68.7489013671875, 91.97723388671875, 0.305254...",2.425783,"[(-5.792812347412109, -32.62266159057617, 0.30..."
4,1,1.jpg,"[182.61886596679688, 57.92866897583008, 0.0728...",1.975177,"[(-2.959381103515625, -16.01549530029297, 0.07..."


In [22]:
type(df.normalized_tuples[0])

list

In [23]:
type(df.normalized_tuples[0][1])

tuple

In [24]:
df['width'] = df.keypoints.apply(get_width)

In [25]:
df['height'] = df.keypoints.apply(get_height)

In [26]:
def get_part_position(bodypart, keypoints, normalize=True):
    tuples = get_3_tuples(keypoints)
    if normalize:
        tuples = normalize_tuples(keypoints)
    try:
        return tuples[coco_format_flip[bodypart]][0:2]
    except:
        print('Something wrong')

In [27]:
get_part_position('Nose',df.keypoints[0])

(-5.166530609130859, -31.30307388305664)

In [28]:
get_part_position('LAnkle',df.keypoints[0])

(11.244792938232422, 29.479595184326172)

In [29]:
import math
math.atan2(-0.1,-1)

-3.0419240010986313

In [30]:
v1= tuple(np.subtract(get_part_position('Nose',df.keypoints[0]),get_part_position('LHip',df.keypoints[0])))

In [31]:
v2= tuple(np.subtract(get_part_position('LHip',df.keypoints[0]),get_part_position('LAnkle',df.keypoints[0])))

In [32]:
v1,v2

((-5.4704437255859375, -30.391342163085938),
 (-10.940879821777344, -30.391326904296875))

In [33]:
def get_angle_vector(v1,v2):
    x1=v1[0]
    y1=v1[1]
    x2=v2[0]
    y2=v2[1]
    angle = 180*(math.atan2(y1,x1)-math.atan2(y2,x2))/3.1417
    return abs(angle)

In [34]:
get_angle_vector(v1,v2)

9.594574456044525

In [35]:
def get_angle_at_hip(keypoints):
    nose = get_part_position('Nose',keypoints)
    LHip = get_part_position('LHip',keypoints)
    LAnkle = get_part_position('LAnkle',keypoints)
    RHip = get_part_position('RHip',keypoints)
    RAnkle = get_part_position('RAnkle',keypoints)
                               
    v1= tuple(np.subtract(nose,LHip))
    v2= tuple(np.subtract(LHip,LAnkle))
    left_angle = get_angle_vector(v1,v2)
                               
    v1= tuple(np.subtract(nose,RHip))
    v2= tuple(np.subtract(RHip,RAnkle))
    right_angle = get_angle_vector(v1,v2)
    return left_angle+right_angle
    

In [36]:
def is_person_standing_straight(keypoints):
    if get_angle_at_hip(keypoints)<60:
        return 1
    else:
        return 0

In [37]:
is_person_standing_straight(df.keypoints[0])

1

In [38]:
df['is_person_standing'] = df.keypoints.apply(is_person_standing_straight)

In [39]:
df['angle_at_hip'] = df.keypoints.apply(get_angle_at_hip)

In [40]:
df.head()

,category_id,image_id,keypoints,score,normalized_tuples,width,height,is_person_standing,angle_at_hip
0,1,0.jpg,"[71.23208618164062, 93.92021179199219, 0.04932...",2.488578,"[(-5.166530609130859, -31.30307388305664, 0.04...",22.489586,66.253105,1,40.738210
1,1,0.jpg,"[183.46817016601562, 58.444889068603516, 0.059...",2.060894,"[(-2.8624649047851562, -15.922449111938477, 0....",8.587387,35.422977,1,22.757212
2,1,0.jpg,"[62.8689079284668, 63.7376594543457, 0.2037215...",0.670776,"[(-1.2289466857910156, -5.635036468505859, 0.2...",8.057343,13.437439,0,432.083170
3,1,1.jpg,"[68.7489013671875, 91.97723388671875, 0.305254...",2.425783,"[(-5.792812347412109, -32.62266159057617, 0.30...",20.122398,67.684395,1,39.615384
4,1,1.jpg,"[182.61886596679688, 57.92866897583008, 0.0728...",1.975177,"[(-2.959381103515625, -16.01549530029297, 0.07...",8.704071,34.816292,1,32.785944


In [41]:
df.groupby('is_person_standing').count()

,category_id,image_id,keypoints,score,normalized_tuples,width,height,angle_at_hip
is_person_standing,,,,,,,,
0,125,125,125,125,125,125,125,125
1,109,109,109,109,109,109,109,109


In [42]:
df.groupby('is_person_standing').width.mean()

is_person_standing
0     9.230322
1    15.490916
Name: width, dtype: float64

In [43]:
df[df.is_person_standing==1]

,category_id,image_id,keypoints,score,normalized_tuples,width,height,is_person_standing,angle_at_hip
0,1,0.jpg,"[71.23208618164062, 93.92021179199219, 0.04932...",2.488578,"[(-5.166530609130859, -31.30307388305664, 0.04...",22.489586,66.253105,1,40.738210
1,1,0.jpg,"[183.46817016601562, 58.444889068603516, 0.059...",2.060894,"[(-2.8624649047851562, -15.922449111938477, 0....",8.587387,35.422977,1,22.757212
3,1,1.jpg,"[68.7489013671875, 91.97723388671875, 0.305254...",2.425783,"[(-5.792812347412109, -32.62266159057617, 0.30...",20.122398,67.684395,1,39.615384
4,1,1.jpg,"[182.61886596679688, 57.92866897583008, 0.0728...",1.975177,"[(-2.959381103515625, -16.01549530029297, 0.07...",8.704071,34.816292,1,32.785944
6,1,2.jpg,"[67.50032043457031, 92.88613891601562, 0.09231...",2.486094,"[(-4.328220367431641, -32.15251922607422, 0.09...",18.549522,68.014938,1,29.792874
7,1,2.jpg,"[181.15211486816406, 58.22768783569336, 0.0509...",2.117695,"[(-3.2463607788085938, -16.2318058013916, 0.05...",9.378372,35.349262,1,40.480296
9,1,3.jpg,"[66.2460708618164, 92.1902084350586, 0.5831860...",2.540721,"[(-4.1036224365234375, -32.82901382446289, 0.5...",15.783173,68.183327,1,23.819855
10,1,3.jpg,"[180.80462646484375, 58.58100509643555, 0.0126...",2.179123,"[(-3.29193115234375, -16.97943878173828, 0.012...",10.742096,36.037987,1,49.395391
12,1,4.jpg,"[63.815555572509766, 91.91630554199219, 0.4640...",2.584104,"[(-5.214277267456055, -31.937484741210938, 0.4...",16.946407,67.785675,1,27.534967
13,1,4.jpg,"[179.489501953125, 58.595584869384766, 0.03243...",2.293873,"[(-3.0700836181640625, -16.714895248413086, 0....",10.233612,35.476513,1,52.507360


In [44]:
df[df.is_person_standing==1].normalized_tuples[7]

[(-3.2463607788085938, -16.2318058013916, 0.050919249653816),
 (-2.1642379760742188, -17.313928604125977, 0.060436207801103),
 (-3.6070785522460938, -17.674631118774414, 0.028914200142025004),
 (-0.36071014404296875, -16.953218460083008, 0.125199824571609),
 (-3.9677810668945312, -17.313928604125977, 0.011133587919175),
 (2.1642379760742188, -11.903322219848633, 0.6632089614868161),
 (-3.6070785522460938, -12.264028549194336, 0.5628725886344911),
 (4.689186096191406, -5.771306991577148, 0.31685870885849),
 (-4.328483581542969, -6.853429794311523, 0.22568437457084603),
 (-2.5249557495117188, -3.246358871459961, 0.23474836349487302),
 (-4.689186096191406, -3.246358871459961, 0.286800026893615),
 (2.5249404907226562, -0.36070823669433594, 0.534264802932739),
 (-1.4428329467773438, -0.36070823669433594, 0.5383364558219911),
 (-0.36071014404296875, 8.656961441040039, 0.35092347860336304),
 (0.7214126586914062, 9.739084243774414, 0.444020420312881),
 (-1.4428329467773438, 17.674631118774414,

In [45]:
df[df.image_id=='0.jpg']

,category_id,image_id,keypoints,score,normalized_tuples,width,height,is_person_standing,angle_at_hip
0,1,0.jpg,"[71.23208618164062, 93.92021179199219, 0.04932...",2.488578,"[(-5.166530609130859, -31.30307388305664, 0.04...",22.489586,66.253105,1,40.738210
1,1,0.jpg,"[183.46817016601562, 58.444889068603516, 0.059...",2.060894,"[(-2.8624649047851562, -15.922449111938477, 0....",8.587387,35.422977,1,22.757212
2,1,0.jpg,"[62.8689079284668, 63.7376594543457, 0.2037215...",0.670776,"[(-1.2289466857910156, -5.635036468505859, 0.2...",8.057343,13.437439,0,432.083170


In [46]:
df.shape

(234, 9)

In [47]:
df_concat=pd.DataFrame()
for i in os.listdir('../output/badm004/'):
    path = '../output/badm004/'+i+'/alphapose-results.json'
    df = pd.read_json(path)
    df['clip']=str(i)
    df['normalized_tuples'] = df.keypoints.apply(normalize_tuples)
    df['normalized_keypoints'] = df.keypoints.apply(normalize_keypoints)
    df['width'] = df.keypoints.apply(get_width)
    df['height'] = df.keypoints.apply(get_height)
    df['angle_at_hip'] = df.keypoints.apply(get_angle_at_hip)
    df['is_person_standing'] = df.keypoints.apply(is_person_standing_straight)
    df_concat = df_concat.append(df)
    print(df_concat.shape)

(126, 11)
(350, 11)
(634, 11)
(753, 11)
(938, 11)
(1126, 11)
(1344, 11)
(1549, 11)
(1671, 11)
(1822, 11)
(2016, 11)
(2171, 11)
(2379, 11)
(2587, 11)
(2792, 11)
(2942, 11)
(3133, 11)
(3317, 11)
(3448, 11)
(3644, 11)
(3831, 11)
(4021, 11)
(4272, 11)
(4438, 11)
(4659, 11)
(4869, 11)
(5070, 11)
(5304, 11)
(5442, 11)
(5618, 11)
(5835, 11)
(6032, 11)
(6187, 11)
(6333, 11)
(6486, 11)
(6642, 11)
(6831, 11)
(7053, 11)
(7233, 11)
(7408, 11)
(7607, 11)
(7729, 11)
(7937, 11)
(8074, 11)
(8256, 11)
(8429, 11)
(8667, 11)
(8819, 11)
(9063, 11)
(9258, 11)
(9381, 11)
(9563, 11)
(9712, 11)
(10084, 11)
(10205, 11)
(10348, 11)
(10542, 11)
(10737, 11)
(10914, 11)
(11063, 11)
(11307, 11)
(11454, 11)
(11706, 11)
(11889, 11)
(12091, 11)
(12237, 11)
(12458, 11)
(12624, 11)
(12749, 11)
(12963, 11)
(13138, 11)
(13312, 11)
(13526, 11)
(13694, 11)
(13887, 11)
(14241, 11)
(14509, 11)
(14695, 11)
(14836, 11)
(15027, 11)
(15303, 11)
(15433, 11)
(15596, 11)
(15724, 11)
(15894, 11)
(16060, 11)
(16342, 11)
(16501, 11)
(1

In [48]:
df_concat.groupby(['clip']).is_person_standing.mean()

clip
badminton004_clip_00000000    0.375839
badminton004_clip_00000001    0.497041
badminton004_clip_00000002    0.530201
badminton004_clip_00000003    0.528169
badminton004_clip_00000004    0.423841
badminton004_clip_00000005    0.615672
badminton004_clip_00000006    0.792793
badminton004_clip_00000007    0.743935
badminton004_clip_00000008    0.697733
badminton004_clip_00000009    0.623306
badminton004_clip_00000010    0.520170
badminton004_clip_00000011    0.481013
badminton004_clip_00000012    0.462366
badminton004_clip_00000013    0.571429
badminton004_clip_00000014    0.621918
badminton004_clip_00000015    0.632768
badminton004_clip_00000016    0.521978
badminton004_clip_00000017    0.510324
badminton004_clip_00000018    0.577778
badminton004_clip_00000019    0.542857
badminton004_clip_00000020    0.558140
badminton004_clip_00000021    0.426829
badminton004_clip_00000022    0.484076
badminton004_clip_00000023    0.447811
badminton004_clip_00000024    0.555957
badminton004_clip_00

In [49]:
df_concat[df_concat.height>30].groupby(['clip']).is_person_standing.sum()

clip
badminton004_clip_00000000    106
badminton004_clip_00000001    127
badminton004_clip_00000002    148
badminton004_clip_00000003    136
badminton004_clip_00000004    124
badminton004_clip_00000005    160
badminton004_clip_00000006    260
badminton004_clip_00000007    261
badminton004_clip_00000008    255
badminton004_clip_00000009    219
badminton004_clip_00000010    212
badminton004_clip_00000011    172
badminton004_clip_00000012    159
badminton004_clip_00000013    201
badminton004_clip_00000014    202
badminton004_clip_00000015    221
badminton004_clip_00000016    158
badminton004_clip_00000017    165
badminton004_clip_00000018    141
badminton004_clip_00000019    126
badminton004_clip_00000020    120
badminton004_clip_00000021    125
badminton004_clip_00000022    114
badminton004_clip_00000023    114
badminton004_clip_00000024    131
badminton004_clip_00000025     97
badminton004_clip_00000026    100
badminton004_clip_00000027     84
badminton004_clip_00000028     78
badminton

In [50]:
df_concat[df_concat.height>30].groupby(['clip','image_id']).is_person_standing.count()

clip                        image_id
badminton004_clip_00000000  0.jpg       4
                            1.jpg       3
                            10.jpg      3
                            11.jpg      3
                            12.jpg      2
                            13.jpg      3
                            14.jpg      4
                            15.jpg      4
                            16.jpg      4
                            17.jpg      4
                            18.jpg      4
                            19.jpg      3
                            2.jpg       3
                            20.jpg      3
                            21.jpg      3
                            22.jpg      3
                            23.jpg      3
                            24.jpg      3
                            25.jpg      3
                            26.jpg      4
                            27.jpg      3
                            28.jpg      4
                            29.jpg     

In [51]:
#df_concat = df_concat[df_concat.height>40]

In [52]:
res = pd.read_json('../output/badminton004.json')

In [53]:
play_clips = [i.split('\\')[-1].split('.')[0] for i in res.video]

In [54]:
play_clips

['badminton004_clip_00000037',
 'badminton004_clip_00000038',
 'badminton004_clip_00000041',
 'badminton004_clip_00000044',
 'badminton004_clip_00000045',
 'badminton004_clip_00000046',
 'badminton004_clip_00000047',
 'badminton004_clip_00000048',
 'badminton004_clip_00000049',
 'badminton004_clip_00000059',
 'badminton004_clip_00000060',
 'badminton004_clip_00000070',
 'badminton004_clip_00000071',
 'badminton004_clip_00000072',
 'badminton004_clip_00000073',
 'badminton004_clip_00000074',
 'badminton004_clip_00000080',
 'badminton004_clip_00000081',
 'badminton004_clip_00000082',
 'badminton004_clip_00000083',
 'badminton004_clip_00000092',
 'badminton004_clip_00000093',
 'badminton004_clip_00000094',
 'badminton004_clip_00000095',
 'badminton004_clip_00000096',
 'badminton004_clip_00000097',
 'badminton004_clip_00000110',
 'badminton004_clip_00000111',
 'badminton004_clip_00000112',
 'badminton004_clip_00000113',
 'badminton004_clip_00000121',
 'badminton004_clip_00000122',
 'badmin

In [55]:
df_concat['play'] = [1 if i in play_clips else 0 for i in df_concat['clip']  ]

In [56]:
df_concat.groupby('play').image_id.count()

play
0    77911
1    25566
Name: image_id, dtype: int64

In [57]:
df_concat.groupby('play').is_person_standing.mean()

play
0    0.577685
1    0.479387
Name: is_person_standing, dtype: float64

In [58]:
df_concat.groupby('play').angle_at_hip.mean()

play
0    145.626735
1    160.211146
Name: angle_at_hip, dtype: float64

In [59]:
df_concat.groupby(['clip','image_id']).is_person_standing.sum()

clip                        image_id
badminton004_clip_00000000  0.jpg       4
                            1.jpg       1
                            10.jpg      1
                            11.jpg      3
                            12.jpg      1
                            13.jpg      3
                            14.jpg      1
                            15.jpg      4
                            16.jpg      3
                            17.jpg      3
                            18.jpg      2
                            19.jpg      2
                            2.jpg       1
                            20.jpg      2
                            21.jpg      2
                            22.jpg      3
                            23.jpg      2
                            24.jpg      2
                            25.jpg      2
                            26.jpg      2
                            27.jpg      1
                            28.jpg      2
                            29.jpg     

In [60]:
df_concat= df_concat.sort_values(by=['image_id', 'height'],ascending=False).drop_duplicates(['clip','image_id'])

In [61]:
df_concat.shape

(33455, 12)

In [62]:
df_concat

,category_id,image_id,keypoints,score,clip,normalized_tuples,normalized_keypoints,width,height,angle_at_hip,is_person_standing,play
27,1,9.jpg,"[0.143286883831024, 65.2054214477539, 0.171229...",1.180523,badminton004_clip_00000460,"[(-5.014352262020111, -45.44734191894531, 0.17...","[-5.014352262020111, -45.44734191894531, 0.171...",12.676343,98.674942,27.565052,1,0
27,1,9.jpg,"[202.00035095214844, 80.92689514160156, 0.0345...",2.340895,badminton004_clip_00000304,"[(17.910140991210938, -35.820281982421875, 0.0...","[17.910140991210938, -35.820281982421875, 0.03...",35.820282,89.550720,18.940489,1,1
20,1,9.jpg,"[208.26551818847656, 89.14549255371094, 0.0088...",2.621004,badminton004_clip_00000452,"[(-8.013832092285156, -40.73694610595703, 0.00...","[-8.013832092285156, -40.73694610595703, 0.008...",28.048386,88.152084,21.848500,1,0
22,1,9.jpg,"[242.90187072753906, 92.23709106445312, 0.0282...",2.550868,badminton004_clip_00000357,"[(-8.099571228027344, -40.8353271484375, 0.028...","[-8.099571228027344, -40.8353271484375, 0.0282...",40.497849,85.720428,20.597983,1,0
31,1,9.jpg,"[181.20265197753906, 85.64785766601562, 0.0067...",2.770652,badminton004_clip_00000322,"[(2.7348175048828125, -37.94558334350586, 0.00...","[2.7348175048828125, -37.94558334350586, 0.006...",34.185211,85.463020,18.110401,1,0
25,1,9.jpg,"[219.0802764892578, 89.63848876953125, 0.00977...",2.519363,badminton004_clip_00000310,"[(-9.438774108886719, -42.474449157714844, 0.0...","[-9.438774108886719, -42.474449157714844, 0.00...",33.709885,84.948898,13.774442,1,0
20,1,9.jpg,"[93.43312072753906, 82.37445068359375, 0.03546...",2.577521,badminton004_clip_00000390,"[(0.0, -41.71017837524414, 0.035462453961372),...","[0.0, -41.71017837524414, 0.035462453961372, 0...",22.815826,84.846336,13.087566,1,0
31,1,9.jpg,"[70.88877868652344, 92.63822937011719, 0.00388...",2.574538,badminton004_clip_00000346,"[(0.9922542572021484, -41.6747932434082, 0.003...","[0.9922542572021484, -41.6747932434082, 0.0038...",44.320812,84.672600,34.183044,1,0
18,1,9.jpg,"[175.2588348388672, 88.23374938964844, 0.00684...",2.702904,badminton004_clip_00000517,"[(-7.533149719238281, -39.37781524658203, 0.00...","[-7.533149719238281, -39.37781524658203, 0.006...",32.871902,84.234268,16.974829,1,0
19,1,9.jpg,"[69.99473571777344, 98.94719696044922, 0.00450...",2.452987,badminton004_clip_00000474,"[(-11.608394622802734, -36.8644905090332, 0.00...","[-11.608394622802734, -36.8644905090332, 0.004...",28.236641,83.768684,205.390151,0,0


In [63]:
df_concat.groupby('play').angle_at_hip.mean()

play
0    67.134796
1    86.227017
Name: angle_at_hip, dtype: float64

In [64]:
df_concat.sort_values(by=['clip', 'image_id'])

,category_id,image_id,keypoints,score,clip,normalized_tuples,normalized_keypoints,width,height,angle_at_hip,is_person_standing,play
0,1,0.jpg,"[159.79774475097656, 91.02629089355469, 0.3793...",2.151584,badminton004_clip_00000000,"[(-10.749862670898438, -25.08300018310547, 0.3...","[-10.749862670898438, -25.08300018310547, 0.37...",21.499725,53.237381,43.896750,1,0
7,1,1.jpg,"[158.23843383789062, 91.81326293945312, 0.5449...",2.313558,badminton004_clip_00000000,"[(-9.400299072265625, -23.500717163085938, 0.5...","[-9.400299072265625, -23.500717163085938, 0.54...",19.845062,53.268311,39.162981,1,0
54,1,10.jpg,"[146.4514617919922, 91.90973663330078, 0.23391...",2.322378,badminton004_clip_00000000,"[(-11.679512023925781, -23.602352142333984, 0....","[-11.679512023925781, -23.602352142333984, 0.2...",24.332321,53.044456,63.379014,0,0
60,1,11.jpg,"[145.97091674804688, 91.72008514404297, 0.3920...",2.358138,badminton004_clip_00000000,"[(-11.82415771484375, -23.401973724365234, 0.3...","[-11.82415771484375, -23.401973724365234, 0.39...",23.648315,52.716026,57.834683,1,0
66,1,12.jpg,"[144.13218688964844, 92.5622787475586, 0.39180...",2.465988,badminton004_clip_00000000,"[(-12.767265319824219, -23.570331573486328, 0....","[-12.767265319824219, -23.570331573486328, 0.3...",25.534531,52.051155,52.556145,1,0
70,1,13.jpg,"[142.68531799316406, 92.76570892333984, 0.3716...",2.479887,badminton004_clip_00000000,"[(-12.721435546875, -23.241085052490234, 0.371...","[-12.721435546875, -23.241085052490234, 0.3716...",25.442871,53.332176,40.156434,1,0
76,1,14.jpg,"[140.58465576171875, 91.55966186523438, 0.3186...",2.467462,badminton004_clip_00000000,"[(-13.069374084472656, -23.42621612548828, 0.3...","[-13.069374084472656, -23.42621612548828, 0.31...",26.138748,52.770645,60.078157,0,0
81,1,15.jpg,"[138.53323364257812, 91.27532958984375, 0.5168...",2.419088,badminton004_clip_00000000,"[(-13.076400756835938, -23.93228530883789, 0.5...","[-13.076400756835938, -23.93228530883789, 0.51...",26.152802,52.799065,53.376843,1,0
88,1,16.jpg,"[137.46340942382812, 90.9141845703125, 0.49635...",2.405835,badminton004_clip_00000000,"[(-12.334495544433594, -23.913818359375, 0.496...","[-12.334495544433594, -23.913818359375, 0.4963...",24.668991,53.869019,35.900998,1,0
94,1,17.jpg,"[136.29566955566406, 90.92233276367188, 0.6140...",2.481316,badminton004_clip_00000000,"[(-11.503005981445312, -24.00627899169922, 0.6...","[-11.503005981445312, -24.00627899169922, 0.61...",23.006012,54.014114,41.308332,1,0


In [65]:
X_mat = np.vstack( (df_concat.normalized_keypoints))

In [66]:
X_mat.shape

(33455, 51)

In [67]:
y= df_concat.play


In [68]:
len(y)

33455

In [69]:
from sklearn.model_selection import GroupShuffleSplit
gss = GroupShuffleSplit(n_splits=10, test_size=0.2, random_state=13)
data_split = gss.split(X_mat, y, groups = df_concat['clip'])
train_ids, test_ids = next(data_split)

In [70]:
len(train_ids)

26747

In [ ]:
train_ids

array([    0,     1,     4, ..., 33451, 33452, 33454])

In [ ]:
len(test_ids)

6708

In [ ]:
from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X_mat, y)
X_train, X_test, y_train, y_test = X_mat[train_ids],X_mat[test_ids],np.array(y)[train_ids],np.array(y)[test_ids]                                                                                  

In [ ]:
X_train.shape

(26747, 51)

In [ ]:
y_train.shape

(26747,)

In [ ]:
X_test.shape

(6708, 51)

In [ ]:
y_test.shape

(6708,)

In [ ]:
from sklearn.ensemble import RandomForestClassifier as rfc

In [ ]:
clf = rfc()
clf.fit(X_train,y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [ ]:
pred = clf.predict(X_test)
pred_proba = clf.predict_proba(X_test)
pred_proba_1 = pred_proba[:,1]

In [ ]:
len(pred_proba_1)

6708

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,pred)

array([[5241,  207],
       [ 258, 1002]])

In [ ]:
sum([pred[i]==list(y_test)[i] for i in range(len(pred))])

6243

In [ ]:
len(pred)

6708

In [ ]:
pred

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
df_concat=pd.DataFrame()
for i in os.listdir('../output/badm001/'):
    try:
        print(i)
        path = '../output/badm001/'+i+'/alphapose-results.json'
        df = pd.read_json(path)
        df['clip']=str(i)
        df['normalized_tuples'] = df.keypoints.apply(normalize_tuples)
        df['normalized_keypoints'] = df.keypoints.apply(normalize_keypoints)
        df['width'] = df.keypoints.apply(get_width)
        df['height'] = df.keypoints.apply(get_height)
        df['angle_at_hip'] = df.keypoints.apply(get_angle_at_hip)
        df['is_person_standing'] = df.keypoints.apply(is_person_standing_straight)
        df_concat = df_concat.append(df)
        print(df_concat.shape)
    except:
        pass

badminton001_clip_00000793
(746, 11)
badminton001_clip_00000755
(1483, 11)
badminton001_clip_00000187
(2018, 11)
badminton001_clip_00000013
(2589, 11)
badminton001_clip_00000228
(3086, 11)
badminton001_clip_00000136
(3765, 11)
badminton001_clip_00000449
(4512, 11)
badminton001_clip_00000002
(4813, 11)
badminton001_clip_00000108
(5307, 11)
badminton001_clip_00000539
(5968, 11)
badminton001_clip_00000769
(6805, 11)
badminton001_clip_00000132
(7398, 11)
badminton001_clip_00000558
(8054, 11)
badminton001_clip_00000101
(8688, 11)
badminton001_clip_00000716
(9571, 11)
badminton001_clip_00000113
(10177, 11)
badminton001_clip_00000296
(10939, 11)
badminton001_clip_00000815
(11579, 11)
badminton001_clip_00000732
(12225, 11)
badminton001_clip_00000124
(12785, 11)
badminton001_clip_00000572
(13550, 11)
badminton001_clip_00000719
(14412, 11)
badminton001_clip_00000440
(15051, 11)
badminton001_clip_00000452
(15738, 11)
badminton001_clip_00000270
(16383, 11)
badminton001_clip_00000545
(17059, 11)
ba

In [ ]:
df_concat

In [ ]:
df_concat= df_concat.sort_values(by=['image_id', 'height'],ascending=False).drop_duplicates(['clip','image_id'])

In [ ]:
X_mat = np.vstack( (df_concat.normalized_keypoints))

In [ ]:
pred = clf.predict(X_mat)
pred

In [ ]:
df_concat['pred']= pred

In [ ]:
len(pred)

In [ ]:
res = df_concat.groupby(['clip']).pred.mean()
res

In [ ]:
list(res.index[res>0.1])

In [ ]:
act = pd.read_json('../output/badminton001.json')

In [ ]:
act_play_clips = [i.split('\\')[-1].split('.')[0].split('_')[-1] for i in act.video]

In [ ]:
act_play_clips

In [ ]:
sum([1 if i in act_play_clips else 0 for i in [i.split('_')[-1] for i in res.index[res>0.05]]])

In [ ]:
res.index[res>0.05].shape

In [ ]:
sum([1 if i in act_play_clips else 0 for i in [i.split('_')[-1] for i in res.index]])

In [ ]:
res.index.shape

In [ ]:
res.sort_values()

In [ ]:
res.sort_values(ascending=False)

In [ ]:
f= open("clipslist.txt","w+")

for i in list(res.index[res>0.1]):
    print(i)
    try:
        f.write("file "+"../output/clips_badminton001/"+i+".mp4\n")
    except:
        pass
f.close()

#os.system('ffmpeg -f concat -safe 0 -i clipslist.txt -c copy highlights.mp4')

#os.system('rm clipslist.txt')


In [ ]:
!ffmpeg -f concat -safe 0 -i clipslist.txt -c copy highlights.mp4

In [ ]:
res